In [1]:
# resnet.py
# Modified from
# https://github.com/pytorch/vision/blob/release/0.8.0/torchvision/models/resnet.py

import torch
from torch import Tensor
import torch.nn as nn
from torchvision.models.utils import load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Optional


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        # Rename relu to relu1
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        self.skip_add = nn.quantized.FloatFunctional()
        # Remember to use two independent ReLU for layer fusion.
        self.relu2 = nn.ReLU(inplace=True)

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)
        
        # Use FloatFunctional for addition for quantization compatibility
        # out += identity
        out = self.skip_add.add(identity, out)
        out = self.relu2(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.relu2  = nn.ReLU(inplace=True)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.downsample = downsample
        self.stride = stride
        self.skip_add = nn.quantized.FloatFunctional()
        self.relu3 = nn.ReLU(inplace=True)
        

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        # out += identity
        out = self.skip_add.add(identity, out)
        out = self.relu3(out)

        return out


class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)


def resnet34(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet50(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet101(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress,
                   **kwargs)


def resnet152(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)


def resnext50_32x4d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def resnext101_32x8d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)


def wide_resnet50_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.

    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet50_2', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def wide_resnet101_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.

    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet101_2', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)

In [2]:
class QuantizedResNet50(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedResNet50, self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.model_fp32 = model_fp32
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.model_fp32(x)
        x = self.dequant(x)
        return x

In [3]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import json
import numpy as np
from PIL import Image

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225])

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])


#model_fp32 = models.resnet18(pretrained=True).to('cpu')
model_fp32 = resnet50(pretrained=True).to('cpu')
#print(model_fp32)
#print(list(model_fp32.children()))

model_fp32.eval()


fused_model = torch.quantization.fuse_modules(model_fp32, [
    ['conv1', 'bn1', 'relu'],
    ['layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu1'],
    ['layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.relu2'],
    ['layer1.0.conv3', 'layer1.0.bn3'],
    ['layer1.0.downsample.0', 'layer1.0.downsample.1'],
    ['layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu1'],
    ['layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.relu2'],
    ['layer1.1.conv3', 'layer1.1.bn3'],
    ['layer1.2.conv1', 'layer1.2.bn1', 'layer1.2.relu1'],
    ['layer1.2.conv2', 'layer1.2.bn2', 'layer1.2.relu2'],
    ['layer1.2.conv3', 'layer1.2.bn3'],
    ['layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu1'],
    ['layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.relu2'],
    ['layer2.0.conv3', 'layer2.0.bn3'],
    ['layer2.0.downsample.0', 'layer2.0.downsample.1'],
    ['layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu1'],
    ['layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.relu2'],
    ['layer2.1.conv3', 'layer2.1.bn3'],
    ['layer2.2.conv1', 'layer2.2.bn1', 'layer2.2.relu1'],
    ['layer2.2.conv2', 'layer2.2.bn2', 'layer2.2.relu2'],
    ['layer2.2.conv3', 'layer2.2.bn3'],
    ['layer2.3.conv1', 'layer2.3.bn1', 'layer2.3.relu1'],
    ['layer2.3.conv2', 'layer2.3.bn2', 'layer2.3.relu2'],
    ['layer2.3.conv3', 'layer2.3.bn3'],
    ['layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu1'],
    ['layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.relu2'],
    ['layer3.0.conv3', 'layer3.0.bn3'],
    ['layer3.0.downsample.0', 'layer3.0.downsample.1'],
    ['layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu1'],
    ['layer3.1.conv2', 'layer3.1.bn2', 'layer3.1.relu2'],
    ['layer3.1.conv3', 'layer3.1.bn3'],
    ['layer3.2.conv1', 'layer3.2.bn1', 'layer3.2.relu1'],
    ['layer3.2.conv2', 'layer3.2.bn2', 'layer3.2.relu2'],
    ['layer3.2.conv3', 'layer3.2.bn3'],
    ['layer3.3.conv1', 'layer3.3.bn1', 'layer3.3.relu1'],
    ['layer3.3.conv2', 'layer3.3.bn2', 'layer3.3.relu2'],
    ['layer3.3.conv3', 'layer3.3.bn3'],
    ['layer3.4.conv1', 'layer3.4.bn1', 'layer3.4.relu1'],
    ['layer3.4.conv2', 'layer3.4.bn2', 'layer3.4.relu2'],
    ['layer3.4.conv3', 'layer3.4.bn3'],
    ['layer3.5.conv1', 'layer3.5.bn1', 'layer3.5.relu1'],
    ['layer3.5.conv2', 'layer3.5.bn2', 'layer3.5.relu2'],
    ['layer3.5.conv3', 'layer3.5.bn3'],
    ['layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu1'],
    ['layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.relu2'],
    ['layer4.0.conv3', 'layer4.0.bn3'],
    ['layer4.0.downsample.0', 'layer4.0.downsample.1'],
    ['layer4.1.conv1', 'layer4.1.bn1', 'layer4.1.relu1'],
    ['layer4.1.conv2', 'layer4.1.bn2', 'layer4.1.relu2'],
    ['layer4.1.conv3', 'layer4.1.bn3'],
    ['layer4.2.conv1', 'layer4.2.bn1', 'layer4.2.relu1'],
    ['layer4.2.conv2', 'layer4.2.bn2', 'layer4.2.relu2'],
    ['layer4.2.conv3', 'layer4.2.bn3'],
], inplace=True)


#print(list(fused_model.children()))

quantized_model = QuantizedResNet50(model_fp32=fused_model)
#print(list(quantized_model.children()))

quantization_config = torch.quantization.get_default_qconfig("fbgemm")
quantized_model.qconfig = quantization_config
#print(quantized_model.qconfig)

torch.quantization.prepare(quantized_model, inplace=True)

# calibrate_model(model=quantized_model, loader=train_loader, device=cpu_device)
quantized_model.to('cpu')
quantized_model.eval()

for index in range(1000):
    file = './Imagenet/' + str(index) + '.jpg'
    img = Image.open(file)
    img_tensor = preprocess(img)
    img_tensor.unsqueeze_(0)
    _ = quantized_model(Variable(img_tensor).to('cpu'))


quantized_model = torch.quantization.convert(quantized_model, inplace=True)
#print(list(quantized_model.children()))
print(quantized_model)

c:\users\yasuy\appdata\local\programs\python\python39\lib\site-packages\torch\quantization\observer.py:119: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


QuantizedResNet50(
  (quant): Quantize(scale=tensor([0.0375]), zero_point=tensor([57]), dtype=torch.quint8)
  (model_fp32): ResNet(
    (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.018436769023537636, zero_point=0, padding=(3, 3))
    (bn1): Identity()
    (relu): Identity()
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(1, 1), stride=(1, 1), scale=0.010135799646377563, zero_point=0)
        (bn1): Identity()
        (relu1): Identity()
        (conv2): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.011952046304941177, zero_point=0, padding=(1, 1))
        (bn2): Identity()
        (relu2): Identity()
        (conv3): QuantizedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), scale=0.020515695214271545, zero_point=59)
        (bn3): Identity()
        (downsample): Sequential(
   

In [5]:
# INFERENCE
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import json
import numpy as np
from PIL import Image

class_index = json.load(open('imagenet_class_index.json', 'r'))

correct_top1 = 0
correct_top5 = 0
average_time = 0
invalid = 0
invalid_wnid = []

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225])

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])


quantized_model.eval()
for index in range(1000):
    #file = './Imagenet/' + str(index) + '.jpg'
    file = './1000/' + str(index) + '.txt'
    try:
        #img = Image.open(file)
        #img_tensor = preprocess(img)
        #img_tensor.unsqueeze_(0)
        
        img = np.loadtxt(file, dtype='float32')
        img = img.reshape(224, 224, 3)
        img_tensor = torch.from_numpy(img.astype(np.float32))
        img_tensor = img_tensor.unsqueeze(0).permute(0,3,1,2)
        
        start = time.perf_counter()
        outputs = quantized_model(Variable(img_tensor).to('cpu'))
        end = time.perf_counter()
        average_time += end - start
        #print(outputs)

        softmax = torch.nn.Softmax(dim=1)
        softmax_out = softmax(outputs)
        #print(outputs.topk(5))

        for i in range(5):
            imagenet_key = str(softmax_out.topk(5)[1][0][i].item())
            if imagenet_key == str(index):
                if i == 0:
                    correct_top1 += 1
                correct_top5 += 1
        print('Top1: ' + str(correct_top1) + '  Top5: ' + str(correct_top5) + '  All: ' + str(index+1))
    except:
        invalid += 1
        #print(class_index[str(index)][0])
        invalid_wnid.append(class_index[str(index)][0])

print('TOP1 ACCURACY: ' + str(correct_top1) + ' / ' + str(1000-invalid))
print('TOP5 ACCURACY: ' + str(correct_top5) + ' / ' + str(1000-invalid))
print('AVERAGE TIME: ' + str(average_time/1000))
print('INVALID DATA: ' + str(invalid))
print(invalid_wnid)


torch.return_types.topk(
values=tensor([[19.0308, 11.1013,  8.7225,  8.1938,  7.1366]]),
indices=tensor([[  0, 389, 391, 394, 395]]))
Top1: 1  Top5: 1  All: 1
TOP1 ACCURACY: 1 / 1000
TOP5 ACCURACY: 1 / 1000
AVERAGE TIME: 5.398740000009638e-05
INVALID DATA: 0
[]


In [10]:
# SHOW ALL KEY
import numpy as np
for key in quantized_model.state_dict():
    print(key)

quant.scale
quant.zero_point
model_fp32.conv1.weight
model_fp32.conv1.bias
model_fp32.conv1.scale
model_fp32.conv1.zero_point
model_fp32.layer1.0.conv1.weight
model_fp32.layer1.0.conv1.bias
model_fp32.layer1.0.conv1.scale
model_fp32.layer1.0.conv1.zero_point
model_fp32.layer1.0.conv2.weight
model_fp32.layer1.0.conv2.bias
model_fp32.layer1.0.conv2.scale
model_fp32.layer1.0.conv2.zero_point
model_fp32.layer1.0.conv3.weight
model_fp32.layer1.0.conv3.bias
model_fp32.layer1.0.conv3.scale
model_fp32.layer1.0.conv3.zero_point
model_fp32.layer1.0.downsample.0.weight
model_fp32.layer1.0.downsample.0.bias
model_fp32.layer1.0.downsample.0.scale
model_fp32.layer1.0.downsample.0.zero_point
model_fp32.layer1.0.skip_add.scale
model_fp32.layer1.0.skip_add.zero_point
model_fp32.layer1.1.conv1.weight
model_fp32.layer1.1.conv1.bias
model_fp32.layer1.1.conv1.scale
model_fp32.layer1.1.conv1.zero_point
model_fp32.layer1.1.conv2.weight
model_fp32.layer1.1.conv2.bias
model_fp32.layer1.1.conv2.scale
model_fp32

In [16]:
#VIEW PARAMS
import numpy as np
key = 'quant.scale'
print(quantized_model.state_dict()[key].item())
#print(np.array(quantized_model.state_dict()[key].int_repr()))
#print(quantized_model.state_dict()[key].q_per_channel_scales())

0.037455279380083084


In [7]:
# SAVE WEIGHT
import numpy as np
for key in quantized_model.state_dict():
    if key[-6:] == 'weight':
        wfilename = key[11:]
        np.savetxt('./resnet50int8-param/'+wfilename+'.txt', np.ravel(np.array(quantized_model.state_dict()[key].int_repr(), dtype='int8')))
        np.savetxt('./resnet50int8-param/'+wfilename[:-6]+'scale.txt', np.ravel(np.array(quantized_model.state_dict()[key].q_per_channel_scales(), dtype='float32')))
        print(wfilename)

conv1.weight
layer1.0.conv1.weight
layer1.0.conv2.weight
layer1.0.conv3.weight
layer1.0.downsample.0.weight
layer1.1.conv1.weight
layer1.1.conv2.weight
layer1.1.conv3.weight
layer1.2.conv1.weight
layer1.2.conv2.weight
layer1.2.conv3.weight
layer2.0.conv1.weight
layer2.0.conv2.weight
layer2.0.conv3.weight
layer2.0.downsample.0.weight
layer2.1.conv1.weight
layer2.1.conv2.weight
layer2.1.conv3.weight
layer2.2.conv1.weight
layer2.2.conv2.weight
layer2.2.conv3.weight
layer2.3.conv1.weight
layer2.3.conv2.weight
layer2.3.conv3.weight
layer3.0.conv1.weight
layer3.0.conv2.weight
layer3.0.conv3.weight
layer3.0.downsample.0.weight
layer3.1.conv1.weight
layer3.1.conv2.weight
layer3.1.conv3.weight
layer3.2.conv1.weight
layer3.2.conv2.weight
layer3.2.conv3.weight
layer3.3.conv1.weight
layer3.3.conv2.weight
layer3.3.conv3.weight
layer3.4.conv1.weight
layer3.4.conv2.weight
layer3.4.conv3.weight
layer3.5.conv1.weight
layer3.5.conv2.weight
layer3.5.conv3.weight
layer4.0.conv1.weight
layer4.0.conv2.weigh

In [8]:
# SAVE BIAS
import numpy as np

for key in quantized_model.state_dict():
    if key[-4:] == 'bias':
        wfilename = key[11:]
        np.savetxt('./resnet50int8-param/'+wfilename+'.txt', np.ravel(quantized_model.state_dict()[key].to('cpu').detach().numpy().copy()))
        print(wfilename)


conv1.bias
layer1.0.conv1.bias
layer1.0.conv2.bias
layer1.0.conv3.bias
layer1.0.downsample.0.bias
layer1.1.conv1.bias
layer1.1.conv2.bias
layer1.1.conv3.bias
layer1.2.conv1.bias
layer1.2.conv2.bias
layer1.2.conv3.bias
layer2.0.conv1.bias
layer2.0.conv2.bias
layer2.0.conv3.bias
layer2.0.downsample.0.bias
layer2.1.conv1.bias
layer2.1.conv2.bias
layer2.1.conv3.bias
layer2.2.conv1.bias
layer2.2.conv2.bias
layer2.2.conv3.bias
layer2.3.conv1.bias
layer2.3.conv2.bias
layer2.3.conv3.bias
layer3.0.conv1.bias
layer3.0.conv2.bias
layer3.0.conv3.bias
layer3.0.downsample.0.bias
layer3.1.conv1.bias
layer3.1.conv2.bias
layer3.1.conv3.bias
layer3.2.conv1.bias
layer3.2.conv2.bias
layer3.2.conv3.bias
layer3.3.conv1.bias
layer3.3.conv2.bias
layer3.3.conv3.bias
layer3.4.conv1.bias
layer3.4.conv2.bias
layer3.4.conv3.bias
layer3.5.conv1.bias
layer3.5.conv2.bias
layer3.5.conv3.bias
layer4.0.conv1.bias
layer4.0.conv2.bias
layer4.0.conv3.bias
layer4.0.downsample.0.bias
layer4.1.conv1.bias
layer4.1.conv2.bias
l

In [9]:
# SAVE FC PARAM
# model_fp32.fc.scale
# model_fp32.fc.zero_point
# model_fp32.fc._packed_params.dtype
# model_fp32.fc._packed_params._packed_params
layername = 'model_fp32.fc.'

print(quantized_model.state_dict()[layername + 'scale'])
print(quantized_model.state_dict()[layername + 'zero_point'])
print(quantized_model.state_dict()[layername + '_packed_params.dtype'])

fcweight = quantized_model.state_dict()[layername + '_packed_params._packed_params'][0]
print(fcweight)

fcweight_int = quantized_model.state_dict()[layername + '_packed_params._packed_params'][0].int_repr()
print(fcweight_int)

fcweight_scale = quantized_model.state_dict()[layername + '_packed_params._packed_params'][0].q_per_channel_scales()
print(fcweight_scale)

fcbias = quantized_model.state_dict()[layername + '_packed_params._packed_params'][1]
print(fcbias)

np.savetxt('./resnet50int8-param/fc.weight.txt', np.ravel(np.array(fcweight_int, dtype='int8')))
np.savetxt('./resnet50int8-param/fc.scale.txt', np.ravel(np.array(fcweight_scale, dtype='float32')))
np.savetxt('./resnet50int8-param/fc.bias.txt', np.ravel(fcbias.to('cpu').detach().numpy().copy()))

tensor(0.2643)
tensor(31)
torch.qint8
tensor([[-0.0091,  0.0159, -0.0364,  ..., -0.0114,  0.0000, -0.0273],
        [ 0.0110, -0.0276,  0.0000,  ...,  0.0276, -0.0221, -0.0055],
        [-0.0066,  0.0658, -0.0395,  ...,  0.0132, -0.0241, -0.0022],
        ...,
        [-0.0098, -0.0020, -0.0354,  ..., -0.0177,  0.1063,  0.0276],
        [-0.0177, -0.0059, -0.0295,  ..., -0.0206, -0.0059, -0.0147],
        [-0.0226,  0.0226,  0.0806,  ...,  0.0000, -0.0452,  0.0194]],
       size=(1000, 2048), dtype=torch.qint8,
       quantization_scheme=torch.per_channel_affine,
       scale=tensor([0.0023, 0.0028, 0.0022, 0.0021, 0.0022, 0.0023, 0.0022, 0.0019, 0.0022,
        0.0020, 0.0020, 0.0024, 0.0025, 0.0025, 0.0023, 0.0018, 0.0020, 0.0023,
        0.0017, 0.0027, 0.0019, 0.0021, 0.0018, 0.0020, 0.0019, 0.0018, 0.0018,
        0.0020, 0.0017, 0.0021, 0.0018, 0.0018, 0.0018, 0.0019, 0.0020, 0.0020,
        0.0020, 0.0019, 0.0017, 0.0017, 0.0014, 0.0018, 0.0017, 0.0017, 0.0015,
        0.0019, 0

tensor([[ -4,   7, -16,  ...,  -5,   0, -12],
        [  4, -10,   0,  ...,  10,  -8,  -2],
        [ -3,  30, -18,  ...,   6, -11,  -1],
        ...,
        [ -5,  -1, -18,  ...,  -9,  54,  14],
        [ -6,  -2, -10,  ...,  -7,  -2,  -5],
        [ -7,   7,  25,  ...,   0, -14,   6]], dtype=torch.int8)
tensor([0.0023, 0.0028, 0.0022, 0.0021, 0.0022, 0.0023, 0.0022, 0.0019, 0.0022,
        0.0020, 0.0020, 0.0024, 0.0025, 0.0025, 0.0023, 0.0018, 0.0020, 0.0023,
        0.0017, 0.0027, 0.0019, 0.0021, 0.0018, 0.0020, 0.0019, 0.0018, 0.0018,
        0.0020, 0.0017, 0.0021, 0.0018, 0.0018, 0.0018, 0.0019, 0.0020, 0.0020,
        0.0020, 0.0019, 0.0017, 0.0017, 0.0014, 0.0018, 0.0017, 0.0017, 0.0015,
        0.0019, 0.0018, 0.0021, 0.0015, 0.0018, 0.0019, 0.0020, 0.0018, 0.0020,
        0.0019, 0.0019, 0.0024, 0.0023, 0.0023, 0.0016, 0.0018, 0.0024, 0.0022,
        0.0019, 0.0017, 0.0020, 0.0019, 0.0019, 0.0019, 0.0024, 0.0024, 0.0018,
        0.0024, 0.0027, 0.0025, 0.0021, 0.0022, 0.00

tensor([-9.0540e-03, -4.1489e-03, -2.0516e-02, -1.7642e-02,  9.6078e-03,
         5.8760e-03, -1.4448e-02,  7.1637e-04,  1.4057e-03, -1.6386e-03,
        -1.2073e-02, -9.3700e-03, -1.9250e-02, -1.7166e-02, -1.1226e-02,
        -1.6972e-02, -8.1289e-03, -1.3626e-02,  8.5068e-03, -1.7855e-02,
         6.9540e-03,  1.8412e-02, -2.4812e-03,  2.4328e-03, -1.3534e-03,
         4.8874e-03,  1.0521e-02, -8.0455e-04, -6.4342e-03, -1.1446e-02,
        -1.3326e-02, -9.2675e-03, -2.5020e-03, -4.5003e-03,  3.7125e-03,
        -2.4581e-02,  4.4222e-03, -8.4804e-03, -8.9190e-06, -1.7266e-03,
        -4.1818e-03,  2.4226e-03,  9.5379e-03, -6.6336e-03,  2.2586e-03,
         3.6322e-04,  2.7801e-02, -1.1270e-02, -1.5046e-02, -8.3841e-03,
         5.7193e-03, -1.5327e-02,  8.8598e-04,  2.3715e-02,  1.3557e-02,
         6.5433e-03,  1.3621e-02, -9.8292e-03,  7.2298e-03,  1.7554e-02,
         2.0959e-04,  4.3992e-03, -8.7419e-03,  9.7137e-03,  1.1456e-02,
         1.5336e-02, -6.9862e-03, -1.3379e-03,  5.0

In [2]:
# MAKE DDR FILE
import random
import time
import math
import numpy as np
import struct

TOO = 64
TII = 32

def read_params(w_fname, b_fname, s_fname, OCH, ICH, K, ddr, ISCALE, OSCALE):
    global bsfile
    print(w_fname)
    tmp = []
    array = []
    
    for line in open(w_fname).readlines():
        tmp.append(int(float(line)))
    
    for to in range(OCH // TOO):
        for ti in range(ICH // TII):
            for i in range(K):
                for j in range(K):
                    for too in range(TOO // 2):
                        for tii in range(TII):
                            array.append(tmp[(to*TOO + 2*too  )*ICH*K*K + (ti*TII + tii)*K*K + i*K + j])
                            array.append(tmp[(to*TOO + 2*too+1)*ICH*K*K + (ti*TII + tii)*K*K + i*K + j])
    
    for i in range(len(array) // 4):
        b0 = format(array[4*i    ] & 0xff, '08b')
        b1 = format(array[4*i + 1] & 0xff, '08b')
        b2 = format(array[4*i + 2] & 0xff, '08b')
        b3 = format(array[4*i + 3] & 0xff, '08b')
        bdata = b3 + b2 + b1 + b0
        idata = int(bdata, 2)
        ddr.append(idata)
    
    for line in open(b_fname).readlines():
        data = float(line) / OSCALE
        bdata = struct.pack('>f', data)
        idata = int.from_bytes(bdata,'big')
        ddr.append(idata)
            
    for line in open(s_fname).readlines():
        data = float(line) * ISCALE / OSCALE
        bdata = struct.pack('>f', data)
        idata = int.from_bytes(bdata,'big')
        ddr.append(idata)


def read_params_conv1(w_fname, b_fname, s_fname, ISCALE, OSCALE):
    tmp = []
    array = []
    wfile = '\nap_int<8> c1_weight[C1_K*C1_K][C1_OCH*C1_ICH] = {\n'
    
    for line in open(w_fname).readlines():
        tmp.append(int(float(line)))
    
    for i in range(7):
        for j in range(7):
            for too in range(64):
                for tii in range(3):
                    array.append(tmp[too*3*7*7 + tii*7*7 + i*7 + j])
    
    for i in range(len(array)):
        if i % (64*3) == 0:
            wfile += '{'
            wfile += str(array[i])
            wfile += ', '
        elif i % (64*3) == 64*3-1:
            wfile += str(array[i])
            if i == 7*7*64*3 - 1:
                wfile += '}\n};\n\n'
            else:
                wfile += '},\n'
        else:
            wfile += str(array[i])
            wfile += ', '
    
    bo = 0
    wfile += 'bias_t c1_bias[C1_OCH] = {'
    for line in open(b_fname).readlines():
        data = float(line) / OSCALE
        wfile += str(data)
        if bo == 64 - 1:
            wfile += '};\n\n'
        else:
            wfile += ', '
        bo += 1
    
    bo = 0
    wfile += 'scale_t c1_scale[C1_OCH] = {'
    for line in open(s_fname).readlines():
        data = float(line) * ISCALE / OSCALE
        wfile += str(data)
        if bo == 64 - 1:
            wfile += '};\n\n'
        else:
            wfile += ', '
        bo += 1
    
    f = open('./resnet50int8-param/c1_weight.h', 'w')
    f.write(wfile)
    f.close()


def read_params_fc(w_fname, b_fname, s_fname, ddr, ISCALE):
    array = []
    
    for line in open(w_fname).readlines():
        array.append(int(float(line)))
    
    for i in range(len(array) // 4):
        b0 = format(array[4*i    ] & 0xff, '08b')
        b1 = format(array[4*i + 1] & 0xff, '08b')
        b2 = format(array[4*i + 2] & 0xff, '08b')
        b3 = format(array[4*i + 3] & 0xff, '08b')
        bdata = b3 + b2 + b1 + b0
        idata = int(bdata, 2)
        ddr.append(idata)
    
    for line in open(b_fname).readlines():
        data = float(line)
        bdata = struct.pack('>f', data)
        idata = int.from_bytes(bdata,'big')
        ddr.append(idata)
            
    for line in open(s_fname).readlines():
        data = float(line) * ISCALE
        bdata = struct.pack('>f', data)
        idata = int.from_bytes(bdata,'big')
        ddr.append(idata)


def save_ddr(fname, array):
    f = open(fname, 'w')
    for i in range(len(array)):
        f.write(str(array[i]) + "\n")
    f.close()


#####################################################################
ddr0 = []
ddr1 = []
ddr2 = []
ddr3 = []

read_params_conv1('./resnet50int8-param/conv1.weight.txt', 
                  './resnet50int8-param/conv1.bias.txt', 
                  './resnet50int8-param/conv1.scale.txt', 0.037455279380083084, 0.018436769023537636)

read_params('./resnet50int8-param/layer1.0.downsample.0.weight.txt',
            './resnet50int8-param/layer1.0.downsample.0.bias.txt',
            './resnet50int8-param/layer1.0.downsample.0.scale.txt', 256, 64, 1, ddr0, 0.018436769023537636, 0.032741695642471313)

read_params('./resnet50int8-param/layer1.0.conv1.weight.txt',
            './resnet50int8-param/layer1.0.conv1.bias.txt',
            './resnet50int8-param/layer1.0.conv1.scale.txt', 64, 64, 1, ddr0, 0.018436769023537636, 0.010135799646377563)

read_params('./resnet50int8-param/layer1.0.conv2.weight.txt',
            './resnet50int8-param/layer1.0.conv2.bias.txt',
            './resnet50int8-param/layer1.0.conv2.scale.txt', 64, 64, 3, ddr0, 0.010135799646377563, 0.011952046304941177)

read_params('./resnet50int8-param/layer1.0.conv3.weight.txt',
            './resnet50int8-param/layer1.0.conv3.bias.txt',
            './resnet50int8-param/layer1.0.conv3.scale.txt', 256, 64, 1, ddr0, 0.011952046304941177, 0.020515695214271545)

read_params('./resnet50int8-param/layer1.1.conv1.weight.txt',
            './resnet50int8-param/layer1.1.conv1.bias.txt',
            './resnet50int8-param/layer1.1.conv1.scale.txt', 64, 256, 1, ddr0, 0.035102978348731995, 0.00922879483550787)

read_params('./resnet50int8-param/layer1.1.conv2.weight.txt',
            './resnet50int8-param/layer1.1.conv2.bias.txt',
            './resnet50int8-param/layer1.1.conv2.scale.txt', 64, 64, 3, ddr0, 0.00922879483550787, 0.015914831310510635)

read_params('./resnet50int8-param/layer1.1.conv3.weight.txt',
            './resnet50int8-param/layer1.1.conv3.bias.txt',
            './resnet50int8-param/layer1.1.conv3.scale.txt', 256, 64, 1, ddr0, 0.015914831310510635, 0.016127794981002808)

read_params('./resnet50int8-param/layer1.2.conv1.weight.txt',
            './resnet50int8-param/layer1.2.conv1.bias.txt',
            './resnet50int8-param/layer1.2.conv1.scale.txt', 64, 256, 1, ddr1, 0.022906972095370293, 0.009117220528423786)

read_params('./resnet50int8-param/layer1.2.conv2.weight.txt',
            './resnet50int8-param/layer1.2.conv2.bias.txt',
            './resnet50int8-param/layer1.2.conv2.scale.txt', 64, 64, 3, ddr1, 0.009117220528423786, 0.019783396273851395)

read_params('./resnet50int8-param/layer1.2.conv3.weight.txt',
            './resnet50int8-param/layer1.2.conv3.bias.txt',
            './resnet50int8-param/layer1.2.conv3.scale.txt', 256, 64, 1, ddr1, 0.019783396273851395, 0.02355561964213848)

read_params('./resnet50int8-param/layer2.0.downsample.0.weight.txt',
            './resnet50int8-param/layer2.0.downsample.0.bias.txt',
            './resnet50int8-param/layer2.0.downsample.0.scale.txt', 512, 256, 1, ddr1, 0.02851303294301033, 0.021553104743361473)

read_params('./resnet50int8-param/layer2.0.conv1.weight.txt',
            './resnet50int8-param/layer2.0.conv1.bias.txt',
            './resnet50int8-param/layer2.0.conv1.scale.txt', 128, 256, 1, ddr1, 0.02851303294301033, 0.011866655200719833)

read_params('./resnet50int8-param/layer2.0.conv2.weight.txt',
            './resnet50int8-param/layer2.0.conv2.bias.txt',
            './resnet50int8-param/layer2.0.conv2.scale.txt', 128, 128, 3, ddr1, 0.011866655200719833, 0.009378612972795963)

read_params('./resnet50int8-param/layer2.0.conv3.weight.txt',
            './resnet50int8-param/layer2.0.conv3.bias.txt',
            './resnet50int8-param/layer2.0.conv3.scale.txt', 512, 128, 1, ddr1, 0.009378612972795963, 0.02354905940592289)

read_params('./resnet50int8-param/layer2.1.conv1.weight.txt',
            './resnet50int8-param/layer2.1.conv1.bias.txt',
            './resnet50int8-param/layer2.1.conv1.scale.txt', 128, 512, 1, ddr1, 0.028694937005639076, 0.006868576630949974)

read_params('./resnet50int8-param/layer2.1.conv2.weight.txt',
            './resnet50int8-param/layer2.1.conv2.bias.txt',
            './resnet50int8-param/layer2.1.conv2.scale.txt', 128, 128, 3, ddr1, 0.006868576630949974, 0.010652955621480942)

read_params('./resnet50int8-param/layer2.1.conv3.weight.txt',
            './resnet50int8-param/layer2.1.conv3.bias.txt',
            './resnet50int8-param/layer2.1.conv3.scale.txt', 512, 128, 1, ddr1, 0.010652955621480942, 0.030405322089791298)

read_params('./resnet50int8-param/layer2.2.conv1.weight.txt',
            './resnet50int8-param/layer2.2.conv1.bias.txt',
            './resnet50int8-param/layer2.2.conv1.scale.txt', 128, 512, 1, ddr1, 0.030345192179083824, 0.009477490559220314)

read_params('./resnet50int8-param/layer2.2.conv2.weight.txt',
            './resnet50int8-param/layer2.2.conv2.bias.txt',
            './resnet50int8-param/layer2.2.conv2.scale.txt', 128, 128, 3, ddr1, 0.009477490559220314, 0.00895635038614273)

read_params('./resnet50int8-param/layer2.2.conv3.weight.txt',
            './resnet50int8-param/layer2.2.conv3.bias.txt',
            './resnet50int8-param/layer2.2.conv3.scale.txt', 512, 128, 1, ddr1, 0.00895635038614273, 0.018680300563573837)

read_params('./resnet50int8-param/layer2.3.conv1.weight.txt',
            './resnet50int8-param/layer2.3.conv1.bias.txt',
            './resnet50int8-param/layer2.3.conv1.scale.txt', 128, 512, 1, ddr1, 0.027333330363035202, 0.0100365299731493)

read_params('./resnet50int8-param/layer2.3.conv2.weight.txt',
            './resnet50int8-param/layer2.3.conv2.bias.txt',
            './resnet50int8-param/layer2.3.conv2.scale.txt', 128, 128, 3, ddr1, 0.0100365299731493, 0.008711608126759529)

read_params('./resnet50int8-param/layer2.3.conv3.weight.txt',
            './resnet50int8-param/layer2.3.conv3.bias.txt',
            './resnet50int8-param/layer2.3.conv3.scale.txt', 512, 128, 1, ddr1, 0.008711608126759529, 0.016849905252456665)

read_params('./resnet50int8-param/layer3.0.downsample.0.weight.txt',
            './resnet50int8-param/layer3.0.downsample.0.bias.txt',
            './resnet50int8-param/layer3.0.downsample.0.scale.txt', 1024, 512, 1, ddr1, 0.02557417005300522, 0.016776524484157562)

read_params('./resnet50int8-param/layer3.0.conv1.weight.txt',
            './resnet50int8-param/layer3.0.conv1.bias.txt',
            './resnet50int8-param/layer3.0.conv1.scale.txt', 256, 512, 1, ddr1, 0.02557417005300522, 0.01329004392027855)

read_params('./resnet50int8-param/layer3.0.conv2.weight.txt',
            './resnet50int8-param/layer3.0.conv2.bias.txt',
            './resnet50int8-param/layer3.0.conv2.scale.txt', 256, 256, 3, ddr1, 0.01329004392027855, 0.010932079516351223)

read_params('./resnet50int8-param/layer3.0.conv3.weight.txt',
            './resnet50int8-param/layer3.0.conv3.bias.txt',
            './resnet50int8-param/layer3.0.conv3.scale.txt', 1024, 256, 1, ddr1, 0.010932079516351223, 0.021548030897974968)

read_params('./resnet50int8-param/layer3.1.conv1.weight.txt',
            './resnet50int8-param/layer3.1.conv1.bias.txt',
            './resnet50int8-param/layer3.1.conv1.scale.txt', 256, 1024, 1, ddr1, 0.024471191689372063, 0.010810386389493942)

read_params('./resnet50int8-param/layer3.1.conv2.weight.txt',
            './resnet50int8-param/layer3.1.conv2.bias.txt',
            './resnet50int8-param/layer3.1.conv2.scale.txt', 256, 256, 3, ddr1, 0.010810386389493942, 0.010484698228538036)

read_params('./resnet50int8-param/layer3.1.conv3.weight.txt',
            './resnet50int8-param/layer3.1.conv3.bias.txt',
            './resnet50int8-param/layer3.1.conv3.scale.txt', 1024, 256, 1, ddr1, 0.010484698228538036, 0.027499347925186157)

read_params('./resnet50int8-param/layer3.2.conv1.weight.txt',
            './resnet50int8-param/layer3.2.conv1.bias.txt',
            './resnet50int8-param/layer3.2.conv1.scale.txt', 256, 1024, 1, ddr2, 0.02869166061282158, 0.008823418989777565)

read_params('./resnet50int8-param/layer3.2.conv2.weight.txt',
            './resnet50int8-param/layer3.2.conv2.bias.txt',
            './resnet50int8-param/layer3.2.conv2.scale.txt', 256, 256, 3, ddr2, 0.008823418989777565, 0.00716364523395896)

read_params('./resnet50int8-param/layer3.2.conv3.weight.txt',
            './resnet50int8-param/layer3.2.conv3.bias.txt',
            './resnet50int8-param/layer3.2.conv3.scale.txt', 1024, 256, 1, ddr2, 0.00716364523395896, 0.013658547773957253)

read_params('./resnet50int8-param/layer3.3.conv1.weight.txt',
            './resnet50int8-param/layer3.3.conv1.bias.txt',
            './resnet50int8-param/layer3.3.conv1.scale.txt', 256, 1024, 1, ddr2, 0.023126672953367233, 0.00954610388725996)

read_params('./resnet50int8-param/layer3.3.conv2.weight.txt',
            './resnet50int8-param/layer3.3.conv2.bias.txt',
            './resnet50int8-param/layer3.3.conv2.scale.txt', 256, 256, 3, ddr2, 0.00954610388725996, 0.007442679721862078)

read_params('./resnet50int8-param/layer3.3.conv3.weight.txt',
            './resnet50int8-param/layer3.3.conv3.bias.txt',
            './resnet50int8-param/layer3.3.conv3.scale.txt', 1024, 256, 1, ddr2, 0.007442679721862078, 0.018794028088450432)

read_params('./resnet50int8-param/layer3.4.conv1.weight.txt',
            './resnet50int8-param/layer3.4.conv1.bias.txt',
            './resnet50int8-param/layer3.4.conv1.scale.txt', 256, 1024, 1, ddr2, 0.025851933285593987, 0.010821557603776455)

read_params('./resnet50int8-param/layer3.4.conv2.weight.txt',
            './resnet50int8-param/layer3.4.conv2.bias.txt',
            './resnet50int8-param/layer3.4.conv2.scale.txt', 256, 256, 3, ddr2, 0.010821557603776455, 0.008071394637227058)

read_params('./resnet50int8-param/layer3.4.conv3.weight.txt',
            './resnet50int8-param/layer3.4.conv3.bias.txt',
            './resnet50int8-param/layer3.4.conv3.scale.txt', 1024, 256, 1, ddr2, 0.008071394637227058, 0.01842919923365116)

read_params('./resnet50int8-param/layer3.5.conv1.weight.txt',
            './resnet50int8-param/layer3.5.conv1.bias.txt',
            './resnet50int8-param/layer3.5.conv1.scale.txt', 256, 1024, 1, ddr2, 0.026974745094776154, 0.014104763977229595)

read_params('./resnet50int8-param/layer3.5.conv2.weight.txt',
            './resnet50int8-param/layer3.5.conv2.bias.txt',
            './resnet50int8-param/layer3.5.conv2.scale.txt', 256, 256, 3, ddr2, 0.014104763977229595, 0.015504616312682629)

read_params('./resnet50int8-param/layer3.5.conv3.weight.txt',
            './resnet50int8-param/layer3.5.conv3.bias.txt',
            './resnet50int8-param/layer3.5.conv3.scale.txt', 1024, 256, 1, ddr2, 0.015504616312682629, 0.026358334347605705)

read_params('./resnet50int8-param/layer4.0.downsample.0.weight.txt',
            './resnet50int8-param/layer4.0.downsample.0.bias.txt',
            './resnet50int8-param/layer4.0.downsample.0.scale.txt', 2048, 1024, 1, ddr2, 0.032445620745420456, 0.05205056816339493)

read_params('./resnet50int8-param/layer4.0.conv1.weight.txt',
            './resnet50int8-param/layer4.0.conv1.bias.txt',
            './resnet50int8-param/layer4.0.conv1.scale.txt', 512, 1024, 1, ddr2, 0.032445620745420456, 0.012715116143226624)

read_params('./resnet50int8-param/layer4.0.conv2.weight.txt',
            './resnet50int8-param/layer4.0.conv2.bias.txt',
            './resnet50int8-param/layer4.0.conv2.scale.txt', 512, 512, 3, ddr2, 0.012715116143226624, 0.01450834795832634)

read_params('./resnet50int8-param/layer4.0.conv3.weight.txt',
            './resnet50int8-param/layer4.0.conv3.bias.txt',
            './resnet50int8-param/layer4.0.conv3.scale.txt', 2048, 512, 1, ddr2, 0.01450834795832634, 0.056098680943250656)

read_params('./resnet50int8-param/layer4.1.conv1.weight.txt',
            './resnet50int8-param/layer4.1.conv1.bias.txt',
            './resnet50int8-param/layer4.1.conv1.scale.txt', 512, 2048, 1, ddr3, 0.07223725318908691, 0.012333572842180729)

read_params('./resnet50int8-param/layer4.1.conv2.weight.txt',
            './resnet50int8-param/layer4.1.conv2.bias.txt',
            './resnet50int8-param/layer4.1.conv2.scale.txt', 512, 512, 3, ddr3, 0.012333572842180729, 0.007715554907917976)

read_params('./resnet50int8-param/layer4.1.conv3.weight.txt',
            './resnet50int8-param/layer4.1.conv3.bias.txt',
            './resnet50int8-param/layer4.1.conv3.scale.txt', 2048, 512, 1, ddr3, 0.007715554907917976, 0.10453417897224426)

read_params('./resnet50int8-param/layer4.2.conv1.weight.txt',
            './resnet50int8-param/layer4.2.conv1.bias.txt',
            './resnet50int8-param/layer4.2.conv1.scale.txt', 512, 2048, 1, ddr3, 0.08634493499994278, 0.011701620183885098)

read_params('./resnet50int8-param/layer4.2.conv2.weight.txt',
            './resnet50int8-param/layer4.2.conv2.bias.txt',
            './resnet50int8-param/layer4.2.conv2.scale.txt', 512, 512, 3, ddr3, 0.011701620183885098, 0.008856935426592827)

read_params('./resnet50int8-param/layer4.2.conv3.weight.txt',
            './resnet50int8-param/layer4.2.conv3.bias.txt',
            './resnet50int8-param/layer4.2.conv3.scale.txt', 2048, 512, 1, ddr3, 0.008856935426592827, 0.11785325407981873)

read_params_fc('./resnet50int8-param/fc.weight.txt',
               './resnet50int8-param/fc.bias.txt',
               './resnet50int8-param/fc.scale.txt', ddr3, 0.15148378908634186)


print('DDR SIZE')
print('ddr0:  ' + str(len(ddr0)))
print('ddr1:  ' + str(len(ddr1)))
print('ddr2:  ' + str(len(ddr2)))
print('ddr3:  ' + str(len(ddr3)))
print('total: ' + str( len(ddr0) + len(ddr1) + len(ddr2) + len(ddr3) ))

save_ddr('./resnet50int8-param/ddr0.txt', ddr0)
save_ddr('./resnet50int8-param/ddr1.txt', ddr1)
save_ddr('./resnet50int8-param/ddr2.txt', ddr2)
save_ddr('./resnet50int8-param/ddr3.txt', ddr3)

./resnet50int8-param/layer1.0.downsample.0.weight.txt
./resnet50int8-param/layer1.0.conv1.weight.txt
./resnet50int8-param/layer1.0.conv2.weight.txt
./resnet50int8-param/layer1.0.conv3.weight.txt
./resnet50int8-param/layer1.1.conv1.weight.txt
./resnet50int8-param/layer1.1.conv2.weight.txt
./resnet50int8-param/layer1.1.conv3.weight.txt
./resnet50int8-param/layer1.2.conv1.weight.txt
./resnet50int8-param/layer1.2.conv2.weight.txt
./resnet50int8-param/layer1.2.conv3.weight.txt
./resnet50int8-param/layer2.0.downsample.0.weight.txt
./resnet50int8-param/layer2.0.conv1.weight.txt
./resnet50int8-param/layer2.0.conv2.weight.txt
./resnet50int8-param/layer2.0.conv3.weight.txt
./resnet50int8-param/layer2.1.conv1.weight.txt
./resnet50int8-param/layer2.1.conv2.weight.txt
./resnet50int8-param/layer2.1.conv3.weight.txt
./resnet50int8-param/layer2.2.conv1.weight.txt
./resnet50int8-param/layer2.2.conv2.weight.txt
./resnet50int8-param/layer2.2.conv3.weight.txt
./resnet50int8-param/layer2.3.conv1.weight.txt